# Feature Engineering & Modeling 

## Feature Engineering

In [68]:
# importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

In [69]:
# Reading in Data
train_df = pd.read_csv('./datasets/clean-train.csv')
test_df = pd.read_csv('./datasets/clean-test.csv')



In [70]:
train_id = train_df['Id']
test_id = test_df['Id']
train_df.drop(['Id'], axis=1, inplace=True)
test_df.drop(['Id'], axis=1, inplace=True)
print(train_df.shape, test_df.shape)

(2047, 79) (878, 78)


In [71]:

train_df.reset_index(drop=True, inplace=True)
# Split features and labels
target_var = train_df['SalePrice'].reset_index(drop=True)
train_features = train_df.drop(['SalePrice'], axis=1)
test_features = test_df

# Combine train and test features in order to apply the feature transformation to both dataframes
ames_df = pd.concat([train_features, test_features]).reset_index(drop=True)
ames_df.shape

(2925, 78)

In [72]:
ames_df.drop(columns=['MS SubClass', 'MS Zoning'], inplace=True)


In [73]:
sale_price = np.log1p(target_var)

In [74]:
ames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Data columns (total 76 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Lot Frontage     2925 non-null   float64
 1   Lot Area         2925 non-null   int64  
 2   Street           2925 non-null   object 
 3   Alley            2925 non-null   object 
 4   Lot Shape        2925 non-null   object 
 5   Land Contour     2925 non-null   object 
 6   Utilities        2925 non-null   object 
 7   Lot Config       2925 non-null   object 
 8   Land Slope       2925 non-null   object 
 9   Neighborhood     2925 non-null   object 
 10  Condition 1      2925 non-null   object 
 11  Condition 2      2925 non-null   object 
 12  Bldg Type        2925 non-null   object 
 13  House Style      2925 non-null   object 
 14  Overall Qual     2925 non-null   int64  
 15  Overall Cond     2925 non-null   int64  
 16  Year Built       2925 non-null   int64  
 17  Year Remod/Add

### Manually Engineering Columns

In [75]:
# Making total sf column
ames_df['Total SF'] = ames_df['Total Bsmt SF'] + \
    ames_df['1st Flr SF'] + ames_df['2nd Flr SF']

# Making home age column
ames_df['Home Age'] = 2010 - ames_df['Year Built']

# Making year since remodel column
ames_df['Year Since Remodel'] = ames_df['Yr Sold'].astype(int) - ames_df['Year Remod/Add'].astype(int)

# Making total home qual/cond column
ames_df['Total Home Qual/Cond'] = ames_df['Overall Qual'] + \
    ames_df['Overall Cond']

# Making total bathrooms column
ames_df['Total Bathrooms'] = ames_df['Full Bath'] + \
    (0.5 * ames_df['Half Bath'])

# Making total porch sf column
ames_df['Total Porch SF'] = (ames_df['Open Porch SF'] +
                             ames_df['3Ssn Porch'] +
                             ames_df['Enclosed Porch'] +
                             ames_df['Screen Porch']
                             )

In [76]:
ames_df.drop(index=1717, inplace=True)
target_var.drop(index=1717, inplace=True)

In [77]:
list(ames_df.columns)

['Lot Frontage',
 'Lot Area',
 'Street',
 'Alley',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Lot Config',
 'Land Slope',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Mas Vnr Area',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin SF 1',
 'BsmtFin Type 2',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Heating',
 'Heating QC',
 'Central Air',
 'Electrical',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'Kitchen Qual',
 'TotRms AbvGrd',
 'Functional',
 'Fireplaces',
 'Fireplace Qu',
 'Garage Type',
 'Garage Yr Blt',
 'Garage Finish',
 'Garage Cars',
 'Garage Area',
 'Garage Qual',
 'Garage Cond',
 'Paved D

#### Dropping Unnecessary Columns

In [78]:
ames_df.drop(columns=['Utilities', 'Street', 'Pool QC', 'Garage Yr Blt'], inplace=True)


### Skewness of All Features

In [79]:
numeric_dtypes = ['int64', 'float64']

quant_cols = []

for i in ames_df.columns:
    if ames_df[i].dtype in numeric_dtypes:
        quant_cols.append(i)
    else:
        pass

In [80]:
quant_vars = pd.DataFrame(ames_df[quant_cols])
quant_vars.skew().sort_values(ascending=False)[:40]

Misc Val                22.221225
Pool Area               17.719169
Lot Area                13.341150
Low Qual Fin SF         12.105521
3Ssn Porch              11.391865
Kitchen AbvGr            4.308723
BsmtFin SF 2             4.135900
Enclosed Porch           4.009814
Screen Porch             3.952700
Bsmt Half Bath           3.952126
Mas Vnr Area             2.598593
Open Porch SF            2.495318
Total Porch SF           2.256715
Wood Deck SF             1.848450
Lot Frontage             1.106015
Gr Liv Area              0.978394
1st Flr SF               0.954891
Bsmt Unf SF              0.923990
2nd Flr SF               0.865288
BsmtFin SF 1             0.832234
Total SF                 0.737130
Fireplaces               0.733101
TotRms AbvGrd            0.704477
Half Bath                0.700828
Bsmt Full Bath           0.617589
Home Age                 0.603233
Overall Cond             0.572271
Year Since Remodel       0.448841
Total Bsmt SF            0.410190
Total Bathroom

#### Insights:

- No strong negative skewness (Skew < -1.0)
 
- Features with Moderate Negative Skew (-0.5 < Skew > 1.0): Year Built

- Features with Strong Positive Skew (Skew > 1.0): MS SubClass, Lot Frontage, Lot Area, Mas Vnr Area, BsmtFin SF 1, BsmtFin SF 2, Total Bsmt SF, 1st Flr SF, Low Qual Fin SF, Gr Liv Area, Bsmt Half Bath, Kitchen AbvGr, Wood Deck SF, Open Porch SF, Enclosed Porch, 3Ssn Porch, Screen Porch, Pool Area, Misc Val      

- Features with Moderate Positive Skew (0.5 > Skew < 1.0):Overall Cond, Bsmt Unf SF, 2nd Flr SF, Bsmt Full Bath, Half Bath, TotRms AbvGrd, Fireplaces

In [81]:
# Creating skewed variable list
skewed_vars = quant_vars.skew()
list(skewed_vars[skewed_vars > 0.5].index)

['Lot Frontage',
 'Lot Area',
 'Overall Cond',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Half Bath',
 'Kitchen AbvGr',
 'TotRms AbvGrd',
 'Fireplaces',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'Total SF',
 'Home Age',
 'Total Porch SF']

In [82]:
skewed_vars.drop(
    labels=["Overall Qual", "Overall Cond",
            "Year Built", "Mo Sold", "Yr Sold"],
    inplace=True,
)

In [83]:
skewed_vars = abs(skewed_vars)

In [84]:
for i in skewed_vars.index:
    ames_df[i] = np.log1p(ames_df[i])

In [85]:
ames_df.head()

,Lot Frontage,Lot Area,Alley,Lot Shape,Land Contour,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Total SF,Home Age,Year Since Remodel,Total Home Qual/Cond,Total Bathrooms,Total Porch SF
0,4.262680,9.511777,none,IR1,Lvl,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,7.603898,Gable,CompShg,HdBoard,Plywood,BrkFace,5.669881,Gd,TA,CBlock,TA,TA,No,GLQ,6.280396,Unf,0.0,5.262690,6.587550,GasA,Ex,Y,SBrkr,6.587550,6.626718,0.0,7.299797,0.000000,0.0,1.098612,0.693147,1.386294,0.693147,Gd,1.945910,Typ,0.000000,none,Attchd,RFn,1.098612,6.165418,TA,TA,Y,0.000000,3.806662,0.0,0.0,0.0,0.0,none,none,0.0,3,2010,WD,7.698483,3.555348,1.791759,2.708050,1.252763,3.806662
1,3.784190,9.349493,none,IR1,Lvl,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,7.599902,Gable,CompShg,VinylSd,VinylSd,BrkFace,4.890349,Gd,TA,PConc,Gd,TA,No,GLQ,6.458338,Unf,0.0,5.624018,6.817831,GasA,Ex,Y,SBrkr,6.817831,7.098376,0.0,7.660585,0.693147,0.0,1.098612,0.693147,1.609438,0.693147,Gd,2.197225,Typ,0.693147,TA,Attchd,RFn,1.098612,6.327937,TA,TA,Y,0.000000,4.317488,0.0,0.0,0.0,0.0,none,none,0.0,4,2009,WD,8.018296,2.708050,2.564949,2.564949,1.252763,4.317488
2,4.234107,8.977525,none,Reg,Lvl,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,7.604894,Gable,CompShg,VinylSd,VinylSd,None,0.000000,TA,Gd,CBlock,TA,TA,No,GLQ,6.595781,Unf,0.0,5.789960,6.964136,GasA,TA,Y,SBrkr,6.964136,0.000000,0.0,6.964136,0.693147,0.0,0.693147,0.000000,1.386294,0.693147,Gd,1.791759,Typ,0.000000,none,Detchd,Unf,0.693147,5.509388,TA,TA,Y,0.000000,3.970292,0.0,0.0,0.0,0.0,none,none,0.0,1,2010,WD,7.656810,4.060443,1.386294,2.564949,0.693147,3.970292
3,4.304065,9.190444,none,Reg,Lvl,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,7.604894,Gable,CompShg,VinylSd,VinylSd,None,0.000000,TA,TA,PConc,Gd,TA,No,Unf,0.000000,Unf,0.0,5.953243,5.953243,GasA,Gd,Y,SBrkr,6.613384,6.552508,0.0,7.275865,0.000000,0.0,1.098612,0.693147,1.386294,0.693147,TA,2.079442,Typ,0.000000,none,BuiltIn,Fin,1.098612,5.993961,TA,TA,Y,4.615121,0.000000,0.0,0.0,0.0,0.0,none,none,0.0,4,2010,WD,7.511525,1.609438,1.386294,2.397895,1.252763,0.000000
4,4.418841,9.563529,none,IR1,Lvl,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,7.597898,Gable,CompShg,Wd Sdng,Plywood,None,0.000000,TA,TA,PConc,Fa,Gd,No,Unf,0.000000,Unf,0.0,6.517671,6.517671,GasA,TA,Y,SBrkr,6.723832,6.421622,0.0,7.276556,0.000000,0.0,1.098612,0.000000,1.386294,0.693147,TA,1.945910,Typ,0.000000,none,Detchd,Unf,1.098612,6.184149,TA,TA,N,0.000000,4.094345,0.0,0.0,0.0,0.0,none,none,0.0,3,2010,WD,7.660114,4.709530,2.890372,2.708050,1.098612,4.094345


### Label Encoding

- Some features have a rating, e.g. Fair, Excellent, Poor, etc.  
- Weighting the homes based on feature quality will maintain the ordinal nature and category.

In [86]:
ames_df['Exter Qual'] = ames_df['Exter Qual'].map(
    {'Ex': 2, 'Gd': 1.5, 'TA': 1, 'Fa': 0.5, 'Po': 0})

ames_df['Exter Cond'] = ames_df['Exter Cond'].map(
    {'Ex': 2, 'Gd': 1.5, 'TA': 1, 'Fa': 0.5, 'Po': 0})


ames_df['Bsmt Cond'] = ames_df['Bsmt Cond'].map(
    {'Ex': 2, 'Gd': 1.5, 'TA': 1, 'Fa': 0.5, 'Po': 0, 'none': 0})

ames_df['BsmtFin Type 1'] = ames_df['BsmtFin Type 1'].map(
    {'GLQ': 2, 'ALQ': 1, 'Rec': 0.5, 'BLQ': 0, 'Unf': 0, 'LwQ': 0, 'none': 0})
ames_df['BsmtFin Type 2'] = ames_df['BsmtFin Type 2'].map(
    {'GLQ': 2, 'ALQ': 1, 'Rec': 0.5, 'BLQ': 0, 'Unf': 0, 'LwQ': 0, 'none': 0})

ames_df['Kitchen Qual'] = ames_df['Kitchen Qual'].map(
    {'Ex': 2, 'Gd': 1.5, 'TA': 1, 'Fa': 0, 'Po': 0})

In [87]:
ames_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2924 entries, 0 to 2924
Data columns (total 78 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Lot Frontage          2924 non-null   float64
 1   Lot Area              2924 non-null   float64
 2   Alley                 2924 non-null   object 
 3   Lot Shape             2924 non-null   object 
 4   Land Contour          2924 non-null   object 
 5   Lot Config            2924 non-null   object 
 6   Land Slope            2924 non-null   object 
 7   Neighborhood          2924 non-null   object 
 8   Condition 1           2924 non-null   object 
 9   Condition 2           2924 non-null   object 
 10  Bldg Type             2924 non-null   object 
 11  House Style           2924 non-null   object 
 12  Overall Qual          2924 non-null   int64  
 13  Overall Cond          2924 non-null   int64  
 14  Year Built            2924 non-null   int64  
 15  Year Remod/Add       

In [88]:

ames_df['Yr Sold'] = ames_df['Yr Sold'].astype(str)
ames_df['Mo Sold'] = ames_df['Mo Sold'].astype(str)


In [90]:
train_dummied = pd.get_dummies(ames_df).reset_index(drop=True)

In [91]:
model_df = pd.concat([train_dummied, poly_df], axis=1)

In [92]:

model_df.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Cond,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Total SF,Home Age,Year Since Remodel,Total Home Qual/Cond,Total Bathrooms,Total Porch SF,Alley_Grvl,Alley_Pave,Alley_none,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Gtl,Land Slope_Mod,Land Slope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Artery,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsbShng,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsbShng,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone,Mas Vnr Type_none,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt Qual_Ex,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_Po,Bsmt Qual_TA,Bsmt Qual_none,Bsmt Exposure_Av,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,Bsmt Exposure_none,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Ex,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_N,Central Air_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Fireplace Qu_Ex,Fireplace Qu_Fa,Fireplace Qu_Gd,Fireplace Qu_Po,Fireplace Qu_TA,Fireplace Qu_none,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_none,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf,Garage Finish_none,Garage Qual_Ex,Garage Qual_Fa,Garage Qual_Gd,Garage Qual_Po,Ga

In [93]:
model_df.shape

(2924, 1319)

In [94]:
X_train = model_df.iloc[:len(target_var), :]
X_test = model_df.iloc[len(target_var):, :]
print(X_train.shape, sale_price.shape, X_test.shape)


(2046, 1319) (2047,) (878, 1319)


In [95]:
X_train.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Cond,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Total SF,Home Age,Year Since Remodel,Total Home Qual/Cond,Total Bathrooms,Total Porch SF,Alley_Grvl,Alley_Pave,Alley_none,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Land Slope_Gtl,Land Slope_Mod,Land Slope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Artery,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsbShng,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsbShng,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone,Mas Vnr Type_none,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt Qual_Ex,Bsmt Qual_Fa,Bsmt Qual_Gd,Bsmt Qual_Po,Bsmt Qual_TA,Bsmt Qual_none,Bsmt Exposure_Av,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,Bsmt Exposure_none,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating QC_Ex,Heating QC_Fa,Heating QC_Gd,Heating QC_Po,Heating QC_TA,Central Air_N,Central Air_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Fireplace Qu_Ex,Fireplace Qu_Fa,Fireplace Qu_Gd,Fireplace Qu_Po,Fireplace Qu_TA,Fireplace Qu_none,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_none,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf,Garage Finish_none,Garage Qual_Ex,Garage Qual_Fa,Garage Qual_Gd,Garage Qual_Po,Ga

In [96]:
sale_price.drop(index=sale_price.nlargest(n=1).index, inplace=True)

In [97]:
print(X_train.shape, sale_price.shape, X_test.shape)

(2046, 1319) (2046,) (878, 1319)


## Modeling

In [98]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

In [99]:
cross_val_score(lr, X_train, sale_price).mean()

-214.54159600913437

In [100]:
cross_val_score(lasso, X_train, sale_price).mean()

0.8379586943702659

In [101]:
cross_val_score(ridge, X_train, sale_price).mean()

0.660133573916472

#### Lasso is preforming best.

In [102]:
lasso.fit(X_train, sale_price)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [103]:
lasso.score(X_train, sale_price)

0.8469981305098638

In [104]:
preds = lasso.predict(X_test)

target_var
submission_preds = np.expm1(preds)

submission_preds = pd.Series(submission_preds)

In [105]:
# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_lasso_no_poly.csv', index=None)

In [106]:
lr = LinearRegression()

lr.fit(X_train, sale_price)

lr.fit(X_train, sale_price)

lr_preds = lr.predict(X_test)






In [107]:
submission_preds = np.expm1(lr_preds)

submission_preds = pd.Series(submission_preds)



# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_4_lrnopolys.csv', index=None)

In [108]:
from sklearn.linear_model import ElasticNetCV

In [109]:
sale_price

0       11.779136
1       12.301387
2       11.599112
3       12.066816
4       11.838633
          ...    
2042    12.607369
2043    11.320566
2044    12.083911
2045    11.877576
2046    12.149508
Name: SalePrice, Length: 2046, dtype: float64

In [110]:
enet_alphas = np.linspace(0.5, 1.0, 100)


In [111]:
elastic = ElasticNetCV(alphas=enet_alphas, l1_ratio=0.5, cv=5)

In [112]:
elastic = elastic.fit(X_train, sale_price)

In [113]:
elastic.score(X_train, sale_price)

0.8482050280036965

In [114]:
elastic_preds = elastic.predict(X_test)

In [115]:

submission_preds = np.expm1(elastic_preds)

submission_preds = pd.Series(submission_preds)



# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_2_elasticnopolys.csv', index=None)

In [116]:
from sklearn.linear_model import RidgeCV

In [117]:
ridge_alphas = np.logspace(0.1, 10, 100)

In [118]:
ridge = RidgeCV(alphas=ridge_alphas, scoring='r2', cv=5)

In [119]:
ridge.fit(X_train, sale_price)

RidgeCV(alphas=array([1.25892541e+00, 1.58489319e+00, 1.99526231e+00, 2.51188643e+00,
       3.16227766e+00, 3.98107171e+00, 5.01187234e+00, 6.30957344e+00,
       7.94328235e+00, 1.00000000e+01, 1.25892541e+01, 1.58489319e+01,
       1.99526231e+01, 2.51188643e+01, 3.16227766e+01, 3.98107171e+01,
       5.01187234e+01, 6.30957344e+01, 7.94328235e+01, 1.00000000e+02,
       1.25892541e+02, 1.58489319e+0...
       1.25892541e+08, 1.58489319e+08, 1.99526231e+08, 2.51188643e+08,
       3.16227766e+08, 3.98107171e+08, 5.01187234e+08, 6.30957344e+08,
       7.94328235e+08, 1.00000000e+09, 1.25892541e+09, 1.58489319e+09,
       1.99526231e+09, 2.51188643e+09, 3.16227766e+09, 3.98107171e+09,
       5.01187234e+09, 6.30957344e+09, 7.94328235e+09, 1.00000000e+10]),
        cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring='r2',
        store_cv_values=False)

In [120]:
ridge.score(X_train, sale_price)

0.8609020813431669

In [121]:
ridge_preds = ridge.predict(X_test)

In [122]:
ridge_coefs = pd.Series(ridge.coef_, index=X_train.columns)

In [123]:
ridge_coefs.sort_values(ascending=False)

Overall Qual Garage Area       0.001084
Overall Cond 2nd Flr SF        0.000866
Overall Cond Bsmt Unf SF       0.000861
Garage Area Open Porch SF      0.000828
Total Bsmt SF 2nd Flr SF       0.000805
                                 ...   
BsmtFin SF 1 Total Porch SF   -0.000814
BsmtFin SF 2 Total Porch SF   -0.000874
BsmtFin SF 2 Wood Deck SF     -0.001034
BsmtFin SF 2 Screen Porch     -0.001085
Overall Cond BsmtFin SF 1     -0.001507
Length: 1319, dtype: float64

In [124]:

submission_preds = np.expm1(ridge_preds)

submission_preds = pd.Series(submission_preds)



# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_3_ridgenopolys.csv', index=None)

In [125]:
from sklearn.linear_model import LassoCV

In [126]:
l_alphas = np.logspace(-3, 0, 100)

In [127]:
lassocv = LassoCV(alphas=l_alphas, cv=5, max_iter=6000)

In [128]:
lassocv.fit(X_train, sale_price)

/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.636506145143514, tolerance: 0.027338214688743142
  tol, rng, random, positive)
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.825613506655447, tolerance: 0.027338214688743142
  tol, rng, random, positive)
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.7561596760626728, tolerance: 0.027338214688743142
  tol, rng, random, positive)
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/s

LassoCV(alphas=array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.008...
       0.37649358, 0.40370173, 0.43287613, 0.46415888, 0.49770236,
       0.53366992, 0.57223677, 0.61359073, 0.65793322, 0.70548023,
       0.75646333, 0.81113083, 0.869749  , 0.93260335, 1.        ]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=6000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)

In [129]:
lassocv.score(X_train, sale_price)

0.858905617839172

In [130]:
lasso_preds = lassocv.predict(X_test)

In [131]:
submission_preds = np.expm1(lasso_preds)

submission_preds = pd.Series(submission_preds)



# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_4_lassocvnopolys.csv', index=None)

In [57]:
lassocv.

<bound method BaseEstimator.get_params of LassoCV(alphas=array([0.001     , 0.00107227, 0.00114976, 0.00123285, 0.00132194,
       0.00141747, 0.00151991, 0.00162975, 0.00174753, 0.00187382,
       0.00200923, 0.00215443, 0.00231013, 0.00247708, 0.00265609,
       0.00284804, 0.00305386, 0.00327455, 0.00351119, 0.00376494,
       0.00403702, 0.00432876, 0.00464159, 0.00497702, 0.0053367 ,
       0.00572237, 0.00613591, 0.00657933, 0.0070548 , 0.00756463,
       0.008...
       0.37649358, 0.40370173, 0.43287613, 0.46415888, 0.49770236,
       0.53366992, 0.57223677, 0.61359073, 0.65793322, 0.70548023,
       0.75646333, 0.81113083, 0.869749  , 0.93260335, 1.        ]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=6000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
        verbose=False)>

In [53]:
lasso_coefs = pd.Series(lassocv.coef_, index=X_train.columns)

In [145]:
lasso_coefs.sort_values(ascending=False)[:30]

Overall Qual Garage Area           0.006263
Overall Cond Bsmt Unf SF           0.001628
Total Bsmt SF 2nd Flr SF           0.001349
Garage Area Open Porch SF          0.001259
Bsmt Unf SF 2nd Flr SF             0.000990
2nd Flr SF Enclosed Porch          0.000781
Overall Cond 2nd Flr SF            0.000762
Mas Vnr Area Open Porch SF         0.000638
Bsmt Unf SF Enclosed Porch         0.000549
Total Bsmt SF Total Porch SF       0.000409
Year Built Total Home Qual/Cond    0.000358
Mas Vnr Area^2                     0.000345
Bsmt Unf SF Misc Val               0.000298
Overall Cond Garage Area           0.000293
Wood Deck SF Misc Val              0.000286
BsmtFin SF 1^2                     0.000261
Wood Deck SF Enclosed Porch        0.000213
Year Built Total SF                0.000202
Overall Qual Open Porch SF         0.000193
Year Built Gr Liv Area             0.000125
2nd Flr SF Total Porch SF          0.000116
Overall Qual Total Porch SF        0.000106
Mas Vnr Area 2nd Flr SF         

In [135]:
sale_price_sm = np.expm1(sale_price)

In [136]:
import statsmodels.api as sm 


X_train = sm.add_constant(X_train)

In [137]:
X_train.shape

(2046, 1320)

In [139]:
sale_price_sm = np.array(sale_price_sm)

In [140]:
ols = sm.OLS(sale_price_sm, X_train).fit()

In [141]:
ols.summary()

/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/statsmodels/base/model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/derekmcadam/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     11.76
Date:                Thu, 09 Apr 2020   Prob (F-statistic):          5.03e-252
Time:                        21:29:12   Log-Likelihood:                -23105.
No. Observations:                2046   AIC:                         4.853e+04
Df Residuals:                     884   BIC:                         5.507e+04
Df Model:                        1161                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                    9.892e+08   1.01e+09      0.975      0.330      -1e+09    2.98e+09
Lot Frontage                             1.424e+06   5.51e+06      0.258      0.796   -9.39e+06    1.22e+07
Lot Area                                 -6.88e+05   3.79e+06     -0.182      0.856   -8.12e+06    6.74e+06
Overall Qual                            -2.702e+07   2.75e+07     -0.983      0.326    -8.1e+07    2.69e+07
Overall Cond                            -2.573e+07   2.76e+07     -0.933      0.351   -7.99e+07    2.84e+07
Year Built                               2.214e+04   2.88e+05      0.077      0.939   -5.44e+05    5.88e+05
Year Remod/Add                          -9.656e+08      1e+09     -0.965      0.335   -2.93e+09    9.99e+08
Mas Vnr Area                            -8.044e+05   6.04e+05     -1.332      0.183   -1.99e+06    3.81e+05
Exter Qual                              -5.536e+06   8.08e+06     -0.685      0.493   -2.14e+07    1.03e+07
Exter Cond                              -8.731e+06   7.54e+06     -1.159      0.247   -2.35e+07    6.06e+06
Bsmt Cond                               -5.074e+06   9.66e+06     -0.525      0.599    -2.4e+07    1.39e+07
BsmtFin Type 1                             1.5e+06   2.54e+06      0.590      0.556   -3.49e+06    6.49e+06
BsmtFin SF 1                             6.821e+05   7.44e+05      0.916      0.360   -7.79e+05    2.14e+06
BsmtFin Type 2                          -1.861e+07   1.46e+07     -1.276      0.202   -4.72e+07       1e+07
BsmtFin SF 2                             1.116e+06   1.51e+06      0.737      0.461   -1.86e+06    4.09e+06
Bsmt Unf SF                              4.332e+05    1.3e+06      0.333      0.739   -2.12e+06    2.99e+06
Total Bsmt SF                            1.384e+06   8.41e+06      0.164      0.869   -1.51e+07    1.79e+07
1st Flr SF                               2.253e+06   1.68e+07      0.134      0.894   -3.08e+07    3.53e+07
2nd Flr SF                              -5.331e+05   1.57e+06     -0.339      0.734   -3.62e+06    2.55e+06
Low Qual Fin SF                           776.7846    1.1e+04      0.071      0.944   -2.08e+04    2.24e+04
Gr Liv Area                              4.089e+06    3.1e+07      0.132      0.895   -5.67e+07    6.49e+07
Bsmt Full Bath                          -6.782e+05   4.85e+06     -0.140      0.889   -1.02e+07    8.83e+06
Bsmt Half Bath                          -6.293e+06   1.01e+07     -0.624      0.533   -2.61e+07    1.35e+07
Full Bath                                1.855e+07   9.54e+07      0.194      0.846   -1.69e+08    2.06e+08
Half Bath                                6.732e+06   2.82e+07      0.239      0.811   -4.86e+07    6.21e+07
Bedroom AbvGr                            7.558e+06   9.16e+

In [ ]:
ols.

In [58]:
lr = LinearRegression()

lr.fit(X_train, sale_price)

lr.fit(X_train, sale_price)

lr_preds = lr.predict(X_test)




submission_preds = np.expm1(lr_preds)

submission_preds = pd.Series(submission_preds)



# Create columns of Ids and the baseline_array
predict = pd.concat([test_id, submission_preds], axis=1)
predict.columns = ['Id', 'SalePrice']
predict.to_csv('./datasets/kaggle_1.csv', index=None)

In [ ]:
l